## 1.Importing library functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import tensorflow as tf 
# As tensorflow do not have library for cifar10 dataset hence keras has been used to just import the dataset.Rest of the work
# has been done using tensorflow. 
import keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


## 2.One hot encoding labels

In [2]:
from keras.utils import to_categorical
number_of_labels = len(np.unique(y_train))
y_train = to_categorical(y_train,num_classes=number_of_labels)
y_test = to_categorical(y_test,num_classes=number_of_labels)

## 3.Dividing the data    

In [3]:
x_valid=x_train[:5000]
x_train=x_train[5000:]
y_valid=y_train[:5000]
y_train=y_train[5000:]

##  4.Parameters

In [4]:
learning_rate = 0.0001
epochs = 10
batch_size = 128

n_classes = 10  
dropout = 0.75  

## 5.Weights and Biases

In [5]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 16])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 16, 32])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([4*4*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([32])),
    'bc3': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Instructions for updating:
Colocations handled automatically by placer.


## 6.Convolutional Layers

In [6]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

## 7.Max Pooling Layers

In [7]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

## 8.Model

In [8]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 32*32*3 to 16*16*16
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 16*16*16 to 8*8*32
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Layer 3 - 8*8*32 to 4*4*64
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)

    # Fully connected layer - 4*4*64 to 1024
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## 9.Session

In [10]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        i=0
        for batch in range(x_train.shape[0]//batch_size):
            start = i
            end = i+batch_size
            i=i+batch_size
            batch_x = np.array(x_train[start:end])
            batch_y = np.array(y_train[start:end])
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={x: x_valid[:500], y: y_valid[:500], keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1,batch + 1,loss,valid_acc*100))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: x_test[:500],
        y: y_test[:500],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc*100))

Epoch  1, Batch   1 - Loss: 416203456.0000 Validation Accuracy: 13.400000
Epoch  1, Batch   2 - Loss: 329698336.0000 Validation Accuracy: 13.600001
Epoch  1, Batch   3 - Loss: 368905408.0000 Validation Accuracy: 13.600001
Epoch  1, Batch   4 - Loss: 362070400.0000 Validation Accuracy: 13.400000
Epoch  1, Batch   5 - Loss: 347592768.0000 Validation Accuracy: 13.400000
Epoch  1, Batch   6 - Loss: 329079840.0000 Validation Accuracy: 12.600000
Epoch  1, Batch   7 - Loss: 345799808.0000 Validation Accuracy: 12.400000
Epoch  1, Batch   8 - Loss: 339275904.0000 Validation Accuracy: 12.200000
Epoch  1, Batch   9 - Loss: 299881376.0000 Validation Accuracy: 11.800000
Epoch  1, Batch  10 - Loss: 310779520.0000 Validation Accuracy: 11.600000
Epoch  1, Batch  11 - Loss: 327248192.0000 Validation Accuracy: 11.600000
Epoch  1, Batch  12 - Loss: 286945920.0000 Validation Accuracy: 11.600000
Epoch  1, Batch  13 - Loss: 299673344.0000 Validation Accuracy: 11.600000
Epoch  1, Batch  14 - Loss: 273979136.

Epoch  1, Batch 114 - Loss: 53278728.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 115 - Loss: 53933336.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 116 - Loss: 54627588.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 117 - Loss: 49358620.0000 Validation Accuracy: 9.200000
Epoch  1, Batch 118 - Loss: 60222464.0000 Validation Accuracy: 9.000000
Epoch  1, Batch 119 - Loss: 57496688.0000 Validation Accuracy: 9.200000
Epoch  1, Batch 120 - Loss: 51098576.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 121 - Loss: 52345192.0000 Validation Accuracy: 9.200000
Epoch  1, Batch 122 - Loss: 57965928.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 123 - Loss: 56632984.0000 Validation Accuracy: 8.800000
Epoch  1, Batch 124 - Loss: 56591800.0000 Validation Accuracy: 9.200000
Epoch  1, Batch 125 - Loss: 60482328.0000 Validation Accuracy: 9.400000
Epoch  1, Batch 126 - Loss: 59191496.0000 Validation Accuracy: 9.400000
Epoch  1, Batch 127 - Loss: 48923240.0000 Validation Accuracy: 9

Epoch  1, Batch 227 - Loss: 48548652.0000 Validation Accuracy: 13.800000
Epoch  1, Batch 228 - Loss: 43775984.0000 Validation Accuracy: 14.200000
Epoch  1, Batch 229 - Loss: 40046164.0000 Validation Accuracy: 13.800000
Epoch  1, Batch 230 - Loss: 45726568.0000 Validation Accuracy: 13.800000
Epoch  1, Batch 231 - Loss: 47131596.0000 Validation Accuracy: 14.200000
Epoch  1, Batch 232 - Loss: 47390528.0000 Validation Accuracy: 13.400000
Epoch  1, Batch 233 - Loss: 43535556.0000 Validation Accuracy: 14.600000
Epoch  1, Batch 234 - Loss: 41340244.0000 Validation Accuracy: 14.200000
Epoch  1, Batch 235 - Loss: 49792364.0000 Validation Accuracy: 13.800000
Epoch  1, Batch 236 - Loss: 38756936.0000 Validation Accuracy: 13.400000
Epoch  1, Batch 237 - Loss: 44425600.0000 Validation Accuracy: 13.200000
Epoch  1, Batch 238 - Loss: 42069784.0000 Validation Accuracy: 12.400000
Epoch  1, Batch 239 - Loss: 44493788.0000 Validation Accuracy: 12.800001
Epoch  1, Batch 240 - Loss: 46315832.0000 Validatio

Epoch  1, Batch 340 - Loss: 37252212.0000 Validation Accuracy: 17.000000
Epoch  1, Batch 341 - Loss: 39090792.0000 Validation Accuracy: 17.800000
Epoch  1, Batch 342 - Loss: 34177948.0000 Validation Accuracy: 17.000000
Epoch  1, Batch 343 - Loss: 37465668.0000 Validation Accuracy: 17.600000
Epoch  1, Batch 344 - Loss: 34701588.0000 Validation Accuracy: 17.399999
Epoch  1, Batch 345 - Loss: 37187592.0000 Validation Accuracy: 17.399999
Epoch  1, Batch 346 - Loss: 33303656.0000 Validation Accuracy: 17.600000
Epoch  1, Batch 347 - Loss: 37345608.0000 Validation Accuracy: 17.600000
Epoch  1, Batch 348 - Loss: 34487156.0000 Validation Accuracy: 17.000000
Epoch  1, Batch 349 - Loss: 34517980.0000 Validation Accuracy: 16.599999
Epoch  1, Batch 350 - Loss: 37315036.0000 Validation Accuracy: 17.000000
Epoch  1, Batch 351 - Loss: 35789148.0000 Validation Accuracy: 16.599999
Epoch  2, Batch   1 - Loss: 39145544.0000 Validation Accuracy: 16.800000
Epoch  2, Batch   2 - Loss: 39875376.0000 Validatio

Epoch  2, Batch 102 - Loss: 31825526.0000 Validation Accuracy: 19.000000
Epoch  2, Batch 103 - Loss: 30382418.0000 Validation Accuracy: 19.000000
Epoch  2, Batch 104 - Loss: 25684200.0000 Validation Accuracy: 19.599999
Epoch  2, Batch 105 - Loss: 27427662.0000 Validation Accuracy: 19.599999
Epoch  2, Batch 106 - Loss: 28090564.0000 Validation Accuracy: 19.400001
Epoch  2, Batch 107 - Loss: 32000040.0000 Validation Accuracy: 18.600000
Epoch  2, Batch 108 - Loss: 32427758.0000 Validation Accuracy: 19.200000
Epoch  2, Batch 109 - Loss: 26756020.0000 Validation Accuracy: 19.200000
Epoch  2, Batch 110 - Loss: 33981540.0000 Validation Accuracy: 18.600000
Epoch  2, Batch 111 - Loss: 29174912.0000 Validation Accuracy: 19.200000
Epoch  2, Batch 112 - Loss: 33345228.0000 Validation Accuracy: 20.000000
Epoch  2, Batch 113 - Loss: 33619076.0000 Validation Accuracy: 19.200000
Epoch  2, Batch 114 - Loss: 29091920.0000 Validation Accuracy: 18.799999
Epoch  2, Batch 115 - Loss: 26504036.0000 Validatio

...

Epoch  9, Batch 144 - Loss: 2968132.7500 Validation Accuracy: 31.799999
Epoch  9, Batch 145 - Loss: 4013878.0000 Validation Accuracy: 31.600001
Epoch  9, Batch 146 - Loss: 3494341.2500 Validation Accuracy: 31.600001
Epoch  9, Batch 147 - Loss: 3889513.7500 Validation Accuracy: 31.999999
Epoch  9, Batch 148 - Loss: 3482883.0000 Validation Accuracy: 32.200000
Epoch  9, Batch 149 - Loss: 3580181.5000 Validation Accuracy: 31.999999
Epoch  9, Batch 150 - Loss: 3717008.2500 Validation Accuracy: 31.799999
Epoch  9, Batch 151 - Loss: 4146713.7500 Validation Accuracy: 31.799999
Epoch  9, Batch 152 - Loss: 3970784.5000 Validation Accuracy: 31.999999
Epoch  9, Batch 153 - Loss: 3647853.5000 Validation Accuracy: 32.200000
Epoch  9, Batch 154 - Loss: 3652216.2500 Validation Accuracy: 32.200000
Epoch  9, Batch 155 - Loss: 4283396.0000 Validation Accuracy: 32.200000
Epoch  9, Batch 156 - Loss: 3465250.7500 Validation Accuracy: 32.800001
Epoch  9, Batch 157 - Loss: 3728380.2500 Validation Accuracy: 32

Epoch  9, Batch 258 - Loss: 4194401.0000 Validation Accuracy: 29.600000
Epoch  9, Batch 259 - Loss: 3880337.0000 Validation Accuracy: 29.400000
Epoch  9, Batch 260 - Loss: 3400993.2500 Validation Accuracy: 30.800000
Epoch  9, Batch 261 - Loss: 4023076.0000 Validation Accuracy: 29.800001
Epoch  9, Batch 262 - Loss: 3459320.0000 Validation Accuracy: 30.800000
Epoch  9, Batch 263 - Loss: 3735957.0000 Validation Accuracy: 31.000000
Epoch  9, Batch 264 - Loss: 3207723.5000 Validation Accuracy: 31.600001
Epoch  9, Batch 265 - Loss: 3283007.2500 Validation Accuracy: 31.200001
Epoch  9, Batch 266 - Loss: 3574001.2500 Validation Accuracy: 30.800000
Epoch  9, Batch 267 - Loss: 3608190.0000 Validation Accuracy: 31.000000
Epoch  9, Batch 268 - Loss: 3334117.2500 Validation Accuracy: 30.599999
Epoch  9, Batch 269 - Loss: 3473880.5000 Validation Accuracy: 30.800000
Epoch  9, Batch 270 - Loss: 3114339.7500 Validation Accuracy: 31.400001
Epoch  9, Batch 271 - Loss: 3488053.0000 Validation Accuracy: 31

Epoch 10, Batch  21 - Loss: 3611171.0000 Validation Accuracy: 30.199999
Epoch 10, Batch  22 - Loss: 3425753.5000 Validation Accuracy: 29.800001
Epoch 10, Batch  23 - Loss: 3440992.5000 Validation Accuracy: 30.000001
Epoch 10, Batch  24 - Loss: 2636551.0000 Validation Accuracy: 29.800001
Epoch 10, Batch  25 - Loss: 3553434.7500 Validation Accuracy: 29.400000
Epoch 10, Batch  26 - Loss: 2871219.7500 Validation Accuracy: 29.200000
Epoch 10, Batch  27 - Loss: 3428848.5000 Validation Accuracy: 29.200000
Epoch 10, Batch  28 - Loss: 2677683.5000 Validation Accuracy: 29.800001
Epoch 10, Batch  29 - Loss: 2827708.0000 Validation Accuracy: 29.400000
Epoch 10, Batch  30 - Loss: 3697161.0000 Validation Accuracy: 29.600000
Epoch 10, Batch  31 - Loss: 3615168.7500 Validation Accuracy: 28.999999
Epoch 10, Batch  32 - Loss: 3596970.5000 Validation Accuracy: 29.400000
Epoch 10, Batch  33 - Loss: 2758767.5000 Validation Accuracy: 28.999999
Epoch 10, Batch  34 - Loss: 3139770.5000 Validation Accuracy: 29

Epoch 10, Batch 135 - Loss: 2916015.5000 Validation Accuracy: 28.600001
Epoch 10, Batch 136 - Loss: 2781047.7500 Validation Accuracy: 28.600001
Epoch 10, Batch 137 - Loss: 3018841.5000 Validation Accuracy: 28.600001
Epoch 10, Batch 138 - Loss: 2669466.0000 Validation Accuracy: 28.799999
Epoch 10, Batch 139 - Loss: 2922352.5000 Validation Accuracy: 28.600001
Epoch 10, Batch 140 - Loss: 2948074.7500 Validation Accuracy: 28.999999
Epoch 10, Batch 141 - Loss: 3426102.7500 Validation Accuracy: 28.999999
Epoch 10, Batch 142 - Loss: 3085397.5000 Validation Accuracy: 28.799999
Epoch 10, Batch 143 - Loss: 2851869.2500 Validation Accuracy: 28.799999
Epoch 10, Batch 144 - Loss: 2449543.7500 Validation Accuracy: 29.200000
Epoch 10, Batch 145 - Loss: 3146608.2500 Validation Accuracy: 29.400000
Epoch 10, Batch 146 - Loss: 2887805.0000 Validation Accuracy: 29.600000
Epoch 10, Batch 147 - Loss: 3105642.5000 Validation Accuracy: 29.200000
Epoch 10, Batch 148 - Loss: 2565663.0000 Validation Accuracy: 29

Epoch 10, Batch 249 - Loss: 2840557.0000 Validation Accuracy: 28.999999
Epoch 10, Batch 250 - Loss: 3212373.5000 Validation Accuracy: 28.999999
Epoch 10, Batch 251 - Loss: 2521639.5000 Validation Accuracy: 29.200000
Epoch 10, Batch 252 - Loss: 2885941.5000 Validation Accuracy: 28.799999
Epoch 10, Batch 253 - Loss: 3099163.0000 Validation Accuracy: 28.999999
Epoch 10, Batch 254 - Loss: 2798325.0000 Validation Accuracy: 28.799999
Epoch 10, Batch 255 - Loss: 2823602.7500 Validation Accuracy: 28.799999
Epoch 10, Batch 256 - Loss: 2991674.0000 Validation Accuracy: 28.999999
Epoch 10, Batch 257 - Loss: 2910489.5000 Validation Accuracy: 29.200000
Epoch 10, Batch 258 - Loss: 3376095.0000 Validation Accuracy: 28.799999
Epoch 10, Batch 259 - Loss: 3126479.0000 Validation Accuracy: 29.200000
Epoch 10, Batch 260 - Loss: 2902347.5000 Validation Accuracy: 28.999999
Epoch 10, Batch 261 - Loss: 3087490.0000 Validation Accuracy: 29.200000
Epoch 10, Batch 262 - Loss: 2866041.0000 Validation Accuracy: 28